### Raven's Progressive Matrix-Style Online Learning of Dynamics from Trajectories

This notebook example demonstrates the use of the online learning rule from _Eliasmith, C. How to Build a Brain, pg. 140, 148, 149_. In that book, it is used to perform "syntactic generalization" or "rule induction" to solve the Raven's Progressive Matrix (RPM) test of fluid intelligence. We apply this same method here to learn the transformation required to produce some dynamical SSP trajectory.

$$T \leftarrow T - w \left[ T - \left( A^{-1} \circledast B \right) \right]$$

where $w = e^{-dt / \tau}$ is the discretization of a lowpass filter with time-constant $\tau$ and time-step $dt$. $A$ is the SSP before the tranformation, and $B$ is the SSP after the transformation (such that $B = T \circledast A$).

Surprisingly, we find that a time-constant of $\bar{\tau} = \tau / dt = 1$ and just a single example is enough to perfectly learn the trajectory.

This in fact is a consequence of the way the trajectory SSPs are defined. In particular,

$$SSP = \sum_{c} X^{x_c} \circledast Y^{y_c} \circledast C^c$$

is the encoding for the trajectory. The decoding is straightforward due to properties of unitary semantic pointers:

$$X^{x_c} \circledast Y^{y_c} \approx SSP \circledast C^{-c}$$

The insight is that to move from one $c$ to another, the required transformation is simply a binding with the vector:

$$T = C^{-\Delta c}$$

And so we find that the online learning rule arrives at the exact same $T$ (after normalization).

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import HTML
import numpy as np

from ssp.maps import Spatial2D
from ssp.cleanup import Cleanup
from ssp.plots import heatmap_animation, create_gif
from ssp.utils import interpolate, linear_steps

In [ ]:
ssp_map = Spatial2D(dim=2048, scale=5)
ssp_map.build_grid(x_len=2, y_len=2, x_spaces=101, y_spaces=101)

In [ ]:
traj_steps = 25
step_size = 1

In [ ]:
# define some functions to use for generating trajectories
absval = lambda x: np.abs(x)
cosine = lambda x: 0.5 * np.cos(x * np.pi) + 1
circle = lambda x, n: (1 + np.cos(2 * np.pi / n * x) / 2, 1 + np.sin(2 * np.pi / n * x) / 2)

absval_points = [(x, absval(x)) for x in np.linspace(0, ssp_map.x_len, traj_steps)]
cosine_points = [(x, cosine(x)) for x in np.linspace(0, ssp_map.x_len, traj_steps)]
circle_points = [circle(x, traj_steps) for x in range(traj_steps)]

In [ ]:
# encode and decode the trajectories as defined by sample points
enc_cues = linear_steps(traj_steps, stepsize=step_size)
dec_cues = interpolate(enc_cues, n=5)

def encode_decode(trajectory, cleanup=None):
    enc = ssp_map.encode_trajectory(points=trajectory, cues=enc_cues)
    dec = ssp_map.decode_trajectory(enc, cues=dec_cues)
    if cleanup is not None:
        dec = list(map(cleanup, dec))
    return [ssp_map.compute_heatmap(v) for v in dec]

In [ ]:
absval_sims = encode_decode(absval_points)
cosine_sims = encode_decode(cosine_points)
circle_sims = encode_decode(circle_points)

ani = heatmap_animation(
    [absval_sims, cosine_sims, circle_sims], figsize=(12, 4), interval=40)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))

In [ ]:
# Raven's Progressive Matrices (RPM) style online learning rule
# Eliasmith, C. How to Build a Brain, pg. 140
trajectory = circle_points
n_online_samples = 1

# express the update as a discrete-time lowpass filter
tau_bar = 1  # tau / dt
w = np.exp(-1 / tau_bar)  # discretizing using ZOH
assert np.allclose(-1 / np.log(w), tau_bar)

# TODO: duplicates encode_decode function
enc = ssp_map.encode_trajectory(points=trajectory, cues=enc_cues)
dec = ssp_map.decode_trajectory(enc, cues=dec_cues)
sims = list(map(ssp_map.compute_heatmap, dec))

T = ssp_map.voc["Zero"]
for i in range(n_online_samples):
    T = T - w*(T - (~dec[i]) * dec[i+1])
T = T.unitary()

In [ ]:
pred_dec = [dec[0]]
for i in range(len(dec) - 1):
    pred_dec.append(T * pred_dec[-1])
pred_sims = list(map(ssp_map.compute_heatmap, pred_dec))

In [ ]:
ani = heatmap_animation(
    [sims, pred_sims, [s1 - s2 for s1, s2 in zip(sims, pred_sims)]],
    figsize=(12, 4), interval=40)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani, fname="trajectory.gif")))

In [ ]:
# verify zero error
np.allclose(sims, pred_sims)

In [ ]:
# verify that it's learning the transformation that we would algebraically expect
dc = dec_cues[1] - dec_cues[0]
assert np.allclose(np.diff(dec_cues), dc)
np.allclose(T.v, (ssp_map.C ** -dc).v)